In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import pandas_gbq
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr


from time import sleep

import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'

# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials, 
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'     # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



In [ ]:
# kor_index_ohlcv
# kor_index_code_fundamental
# kor_index_list_df

In [12]:
# glob.glob('./data_crawler/kor_index_ohlcv/*')
glob.glob('./data_crawler/cleaning/kor_stock_ohlcv/*')

['./data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_KOSDAQ_20240107.parquet',
 './data_crawler/cleaning/kor_stock_ohlcv/df_raw_anal_total_2_KOSPI_20240107.parquet',
 './data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSDAQ_20240107.parquet',
 './data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSPI_20240107.parquet']

In [10]:
for file_nm in ['kor_index_ohlcv','kor_index_code_fundamental','kor_index_list_df']:
    print(file_nm)
#     df = pd.read_csv(f'./data_crawler/{file_nm}/{file_nm}.csv')
#     table_from_pandas = pa.Table.from_pandas(df,preserve_index = False)
#     pq.write_table(table_from_pandas, f'./data_crawler/{file_nm}/{file_nm}.parquet')

    # Google Storage 적재
    source_file_name = f'./data_crawler/{file_nm}/{file_nm}.parquet'   # GCP에 업로드할 파일 절대경로
    destination_blob_name = f'data_crawler/{file_nm}/{file_nm}.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    

kor_index_ohlcv
kor_index_code_fundamental
kor_index_list_df


In [5]:
glob.glob('./data_crawler/kor_index_code_fundamental/*')

['./data_crawler/kor_index_code_fundamental/kor_index_code_fundamental.csv']

In [6]:
glob.glob('./data_crawler/kor_index_list_df/*')

['./data_crawler/kor_index_list_df/kor_index_list_df.csv']

In [ ]:
/home/shjj08choi4/finance_mlops/data_crawler/cleaning/kor_stock_ohlcv

### ghkrdls

In [13]:
t = pq.read_table('./data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSPI_20240107.parquet')
df = t.to_pandas()

In [15]:
df[df['ticker'] == '005930']['ㅇ']

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name,market,...,MACD_Signal,변화량,상승폭,하락폭,AU,AD,RSI,MA5-20,MA20-60,MA60-120
100533,2023-07-21 00:00:00+00:00,70400,70400,69400,70300.0,16528926,-0.985915,005930,삼성전자,KOSPI,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
100534,2023-07-10 00:00:00+00:00,70000,70400,69200,69500.0,11713926,-0.572246,005930,삼성전자,KOSPI,...,NaN,-800.0,0.0,800.0,NaN,NaN,NaN,NaN,NaN,NaN
100535,2023-05-26 00:00:00+00:00,69800,70400,69500,70300.0,19549511,2.180233,005930,삼성전자,KOSPI,...,NaN,800.0,800.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
100536,2023-02-03 00:00:00+00:00,63900,64000,63000,63800.0,15194598,0.472441,005930,삼성전자,KOSPI,...,NaN,6800.0,6800.0,0.0,1743.030417,1372.614550,55.944449,65.0,NaN,NaN
100537,2023-04-03 00:00:00+00:00,64000,64000,63000,63100.0,11973133,-1.406250,005930,삼성전자,KOSPI,...,NaN,-700.0,0.0,700.0,1588.204691,1312.869157,54.745407,1585.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100777,2023-04-10 00:00:00+00:00,64800,66300,64700,65700.0,18777166,1.076923,005930,삼성전자,KOSPI,...,-1075.460430,0.0,0.0,0.0,2745.035778,2307.057374,54.334623,6845.0,-362.500000,-2612.750000
100778,2023-06-30 00:00:00+00:00,72500,72700,71700,72200.0,11694765,-0.276243,005930,삼성전자,KOSPI,...,-604.197107,6500.0,6500.0,0.0,3013.247508,2142.267561,58.447070,8575.0,-1233.833333,-2271.416667
100779,2023-06-28 00:00:00+00:00,72600,72700,72000,72700.0,8783093,0.137741,005930,삼성전자,KOSPI,...,424.503130,1200.0,1200.0,0.0,2683.871576,1893.587846,58.632340,12015.0,-2986.333333,-1545.583333
100780,2023-11-28 00:00:00+00:00,71400,72700,71300,72700.0,13283081,1.963534,005930,삼성전자,KOSPI,...,927.518787,0.0,0.0,0.0,2492.166464,1758.331572,58.632340,11897.5,-1889.666667,-1148.083333
